In [25]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.typing import NDArray
from scipy import stats
import scipy

In [26]:
class Simulation:
    def __init__(self, transition_matrix) -> None:
        self.transition_matrix = transition_matrix

    def simulate_time(self, rate) -> float:
        return np.random.exponential(scale=1/rate)
    
    def simulate_transition(self, state) -> int:
        probs = np.copy(self.transition_matrix[state])
        probs[state] = 0
        probs /= sum(probs)
        next_state = np.random.choice([i for i in range(len(probs))], p=probs)
        return next_state
    
    def simulate_one(self) -> tuple[list, list, bool]:
        states = [0]
        times = []
        reappeared = False
        while True:
            state = states[-1]
            times.append(self.simulate_time(-self.transition_matrix[state][state]))
            next_state = self.simulate_transition(state)
            if next_state == self.transition_matrix.shape[0] - 1:
                break
            if next_state in (2, 3) and sum(times) > 30.5:
                reappeared = True
            states.append(int(next_state))
        return states, times, reappeared
    
    def simulate(self, num_simulations: int) -> tuple[NDArray, float, list[list[int]]]:
        arr = np.empty(num_simulations)
        reappeared_counter = 0
        all_observations = []
        for i in range(num_simulations):
            states, times, reappeared = self.simulate_one()
            arr[i] = np.sum(times)
            if reappeared:
                reappeared_counter += 1

            current_time = 0
            time_interval = 48
            observations = [0]
            state_idx = 0
            while True:
                current_time += time_interval
                while sum(times[:state_idx+1]) < current_time and state_idx < len(states):
                    state_idx += 1
                if state_idx >= len(states):
                    break
                observations.append(states[state_idx])
            observations.append(4)
            all_observations.append(observations)

        self.times = arr
        return arr, reappeared_counter / num_simulations, all_observations
    
    def Ft(self, t):
        Qs = self.transition_matrix[:-1, :-1]
        p0 = np.zeros(Qs.shape[0])
        p0[0] = 1
        return 1 - p0 @ scipy.linalg.expm(Qs * t) @ np.ones(Qs.shape[0])
    
    def evaluate(self):
        points = np.array([[t, self.Ft(t)] for t in np.linspace(0, 1200, 10_000)])

        n = len(self.times)
        x = np.sort(self.times)
        y = np.arange(1, n+1) / n

        plt.plot(points[:, 0], points[:, 1])
        plt.plot(x, y, marker='.', linestyle='none')
        plt.xlabel('Time (months)')
        plt.ylabel('Cumulative Probability')
        plt.title('Empirical Cumulative Distribution Function of Time to Death')
        plt.grid(True)
        plt.legend(["Theoretical data", "Simulated data"])
        plt.show()

    def plot_survival(self):
        t_values = np.array([t for t in np.linspace(0, 1200, 10_000)])
        N = len(self.times)
        d_t = lambda t: sum(self.times <= t)
        S_t = lambda t: (N - d_t(t)) / N
        survival_func_points = [S_t(t) for t in t_values]
        plt.plot(t_values, survival_func_points)

## Task 12

In [27]:
transition_matrix = np.array([
    [-0.0085, 0.005,  0.0025, 0.0,    0.001],
    [0.0,    -0.014,  0.005,  0.004,  0.005],
    [0.0,     0.0,   -0.008,  0.003,  0.005],
    [0.0,     0.0,    0.0,   -0.009,  0.009],
    [0.0,     0.0,    0.0,    0.0,    0.0  ]
])

sim = Simulation(transition_matrix)
N = 1000
times, reappeared_rate, observations = sim.simulate(N)
observations

[[0, 0, 0, 0, 1, 2, 2, 2, 4],
 [0, 4],
 [0, 0, 0, 0, 1, 2, 2, 2, 4],
 [0, 0, 4],
 [0, 1, 1, 1, 1, 1, 1, 3, 3, 4],
 [0, 1, 1, 1, 1, 1, 1, 4],
 [0, 2, 2, 2, 4],
 [0, 0, 1, 4],
 [0, 1, 4],
 [0, 2, 2, 2, 2, 2, 2, 2, 3, 4],
 [0, 0, 0, 0, 0, 0, 2, 2, 2, 3, 4],
 [0, 3, 3, 3, 3, 4],
 [0, 0, 0, 0, 1, 4],
 [0, 4],
 [0, 0, 0, 2, 4],
 [0, 0, 0, 0, 0, 0, 0, 4],
 [0, 0, 0, 0, 0, 2, 4],
 [0, 0, 4],
 [0, 0, 1, 3, 3, 3, 3, 3, 4],
 [0, 0, 0, 4],
 [0, 0, 0, 0, 1, 4],
 [0, 2, 2, 3, 3, 4],
 [0, 0, 2, 2, 2, 2, 2, 4],
 [0, 0, 0, 1, 2, 2, 4],
 [0, 0, 1, 2, 2, 2, 2, 4],
 [0, 0, 0, 2, 4],
 [0, 0, 0, 0, 4],
 [0, 0, 0, 1, 1, 2, 4],
 [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4],
 [0, 0, 0, 0, 0, 0, 1, 1, 2, 2, 2, 3, 3, 4],
 [0, 0, 0, 0, 2, 2, 3, 3, 3, 3, 4],
 [0, 0, 0, 1, 1, 1, 2, 2, 2, 4],
 [0, 0, 0, 0, 2, 4],
 [0, 2, 2, 2, 4],
 [0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 4],
 [0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4],
 [0, 0, 1, 1, 1, 1, 1, 1, 1, 3, 4],
 [0, 1, 3, 4],
 [0, 0, 0, 0, 0, 1, 4],
 [0, 0, 0, 0, 4],
 [0, 0, 2, 2, 2, 2, 2, 2, 2, 4],
 

## Task 13

In [30]:
def simulate_time(rate) -> float:
    return np.random.exponential(scale=1/rate)

def simulate_transition(state, Q0) -> int:
    probs = np.copy(Q0[state])
    probs[state] = 0
    probs /= sum(probs)
    next_state = np.random.choice([i for i in range(len(probs))], p=probs)
    return next_state

def compute_N_S(Q0, times, states, idx):
    N = np.zeros(Q0.shape, dtype=int)
    S = np.zeros(Q0.shape[0], dtype=float)

    for i in range(idx):
        N[states[i]][states[i+1]] += 1
        S[states[i]] += times[i]
    S[states[idx]] += (48 - sum(times[:idx]))
    S[-1] = 0
    return N, S

def simulate_between_observations(Q0, initial_state, final_state):
    while True:
        states = [initial_state]
        times = []
        while True:
            state = states[-1]
            times.append(simulate_time(-Q0[state][state]))
            next_state = simulate_transition(state, Q0)
            states.append(int(next_state))
            if next_state == Q0.shape[0] - 1:
                break

        if sum(times) < 48:
            if final_state == 4:
                return compute_N_S(Q0, times, states, len(times))
            continue

        idx = 0
        while True:
            time_sum = sum(times[:idx+1])
            if time_sum >= 48:
                break
            idx += 1
        
        if states[idx] == final_state:
            if initial_state == final_state:
                S = np.zeros(Q0.shape[0])
                S[initial_state] += 48
                return np.zeros(Q0.shape, dtype=int), S
            return compute_N_S(Q0, times, states, idx)

Q0 = np.triu(np.ones((5, 5), dtype=float), k=1)
Q0 /= 1000
for i, row in enumerate(Q0):
    row[i] = -sum(row)

while True:
    N_total = np.zeros(Q0.shape, dtype=int)
    S_total = np.zeros(Q0.shape[0], dtype=float)
    for woman_vector in observations:
        for idx in range(len(woman_vector) - 1):
            N, S = simulate_between_observations(Q0, woman_vector[idx], woman_vector[idx+1])
            N_total += N
            S_total += S
    Q1 = np.empty(Q0.shape)
    for i, row in enumerate(S_total[:-1]):
        Q1[i] = N_total[i] / row
        Q1[i][i] = -sum(Q1[i])
    
    dif = np.linalg.norm(Q0-Q1, ord=np.inf)
    print(dif)
    if dif < 10 ** (-3):
        break

    Q0 = np.copy(Q1)

print(Q0)

0.014784480461459776
0.0031323856989132273
0.0015056646745418944
0.0005933380248962679
[[-0.00846156  0.0045354   0.00273308  0.00033846  0.00085462]
 [ 0.         -0.01271127  0.00431614  0.00339125  0.00500388]
 [ 0.          0.         -0.00827183  0.00317769  0.00509413]
 [ 0.          0.          0.         -0.00924291  0.00924291]
 [ 0.          0.          0.          0.          0.        ]]
